# Clustering Consulting Project 

A large technology firm needs your help, they've been hacked! Luckily their forensic engineers have grabbed valuable data about the hacks, including information like session time,locations, wpm typing speed, etc. The forensic engineer relates to you what she has been able to figure out so far, she has been able to grab meta data of each session that the hackers used to connect to their servers. These are the features of the data:

* 'Session_Connection_Time': How long the session lasted in minutes
* 'Bytes Transferred': Number of MB transferred during session
* 'Kali_Trace_Used': Indicates if the hacker was using Kali Linux
* 'Servers_Corrupted': Number of server corrupted during the attack
* 'Pages_Corrupted': Number of pages illegally accessed
* 'Location': Location attack came from (Probably useless because the hackers used VPNs)
* 'WPM_Typing_Speed': Their estimated typing speed based on session logs.


The technology firm has 3 potential hackers that perpetrated the attack. Their certain of the first two hackers but they aren't very sure if the third hacker was involved or not. They have requested your help! Can you help figure out whether or not the third suspect had anything to do with the attacks, or was it just two hackers? It's probably not possible to know for sure, but maybe what you've just learned about Clustering can help!

**One last key fact, the forensic engineer knows that the hackers trade off attacks. Meaning they should each have roughly the same amount of attacks. For example if there were 100 total attacks, then in a 2 hacker situation each should have about 50 hacks, in a three hacker situation each would have about 33 hacks. The engineer believes this is the key element to solving this, but doesn't know how to distinguish this unlabeled data into groups of hackers.**

If only first two hackers were involved => k = 2 (2 distinct hackers)
third hacker were also involved => k = 3 (3 distinct hackers)

AND KEY FACT SUGGEST THAT: which k value makes each cluster counts same? 
ex) if k = 2, two clusters needs to have same count! 

In [1]:
import findspark
findspark.init('/home/quanttraderkim/spark-2.3.1-bin-hadoop2.7')

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('p').getOrCreate()

In [4]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler, VectorIndexer, StandardScaler
from pyspark.ml import Pipeline

In [5]:
df = spark.read.csv('hack_data.csv', inferSchema=True, header=True)
display(df.show())
display(df.printSchema())

+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+
|Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted|            Location|WPM_Typing_Speed|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+
|                    8.0|           391.09|              1|             2.96|            7.0|            Slovenia|           72.37|
|                   20.0|           720.99|              0|             3.04|            9.0|British Virgin Is...|           69.08|
|                   31.0|           356.32|              1|             3.71|            8.0|             Tokelau|           70.58|
|                    2.0|           228.08|              1|             2.48|            8.0|             Bolivia|            70.8|
|                   20.0|            408.5|              0|             3.57

None

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)



None

In [6]:
df.columns

['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed']

In [40]:
assembler = VectorAssembler(inputCols=[
 'Kali_Trace_Used',
 'WPM_Typing_Speed'], outputCol='features')
scaler = StandardScaler(inputCol='features', outputCol='scaled_features')

In [41]:
prep_pipeline = Pipeline(stages=[assembler, scaler])

In [42]:
final_df = prep_pipeline.fit(df).transform(df)

In [43]:
kmeans = KMeans(featuresCol='scaled_features', k=2)

In [44]:
model = kmeans.fit(final_df)

In [45]:
wssse_2 = model.computeCost(final_df)
print(wssse_2)

332.76508509230075


In [46]:
centers = model.clusterCenters()
print(centers)

[array([0.        , 4.30291659]), array([1.99757683, 4.24986033])]


In [47]:
kmeans3 = KMeans(featuresCol='scaled_features', k=3)


In [48]:
model2 = kmeans3.fit(final_df)
wssse_3 = model2.computeCost(final_df)
print(wssse_3)

157.7622525485659


In [56]:
pred = model2.transform(final_df)

In [60]:
pred.groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|   83|
|         2|   88|
|         0|  163|
+----------+-----+



3 hackers were all involved!

## IN ONE BLOCK

In [61]:
# two features that I think work well to distinguish people's unique features
assembler = VectorAssembler(inputCols=[
 'Kali_Trace_Used',
 'WPM_Typing_Speed'], outputCol='features')
scaler = StandardScaler(inputCol='features', outputCol='scaled_features')
prep_pipeline = Pipeline(stages=[assembler, scaler])
final_df = prep_pipeline.fit(df).transform(df)

# model
kmeans_2 = KMeans(featuresCol='scaled_features', k=2)
model_2 = kmeans_2.fit(final_df)
wssse_2 = model_2.computeCost(final_df)
pred_2 = model_2.transform(final_df)
display(pred_2.groupBy('prediction').count().show())

kmeans_3 = KMeans(featuresCol='scaled_features', k=3)
model_3 = kmeans_3.fit(final_df)
wssse_3 = model_3.computeCost(final_df)
pred_3 = model_3.transform(final_df)
display(pred_3.groupBy('prediction').count().show())


print('k=2(two hackers): {}'.format(wssse_2))
print('k=3(three hackers): {}'.format(wssse_3))

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  171|
|         0|  163|
+----------+-----+



None

+----------+-----+
|prediction|count|
+----------+-----+
|         1|   83|
|         2|   88|
|         0|  163|
+----------+-----+



None

k=2(two hackers): 332.76508509230075
k=3(three hackers): 157.7622525485659


In [62]:
# using all features except location
assembler2 = VectorAssembler(inputCols=[
'Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'WPM_Typing_Speed'], outputCol='features')
scaler = StandardScaler(inputCol='features', outputCol='scaled_features')
prep_pipeline2 = Pipeline(stages=[assembler2, scaler])
final_df = prep_pipeline2.fit(df).transform(df)

# model
kmeans_2 = KMeans(featuresCol='scaled_features', k=2)
model_2 = kmeans_2.fit(final_df)
wssse_2 = model_2.computeCost(final_df)
pred_2 = model_2.transform(final_df)
display(pred_2.groupBy('prediction').count().show())

kmeans_3 = KMeans(featuresCol='scaled_features', k=3)
model_3 = kmeans_3.fit(final_df)
wssse_3 = model_3.computeCost(final_df)
pred_3 = model_3.transform(final_df)
display(pred_3.groupBy('prediction').count().show())

print('k=2(two hackers): {}'.format(wssse_2))
print('k=3(three hackers): {}'.format(wssse_3))

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         0|  167|
+----------+-----+



None

+----------+-----+
|prediction|count|
+----------+-----+
|         1|   84|
|         2|   83|
|         0|  167|
+----------+-----+



None

k=2(two hackers): 601.7707512676716
k=3(three hackers): 434.1492898715845


1) K = 2 : perfectly even! -> 2 hackers!
2) K = 3: one cluster divided into two clusters, the other stays the same
it should have been just 2 hackers!